In [1]:
from fastai.vision.all import *
from fastcore.parallel import *
import fastai
import pandas as pd
from pathlib import Path

import torch
from torch.utils.data import Dataset

 

# Set paths
path = Path("/kaggle/input/isic-2024-challenge/")
train_metadata_path = path / 'train-metadata.csv'
images_path = path / 'train-image/image/'

# Preparing metadata file and merge with images